In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data=pd.read_csv("Train.csv")
data.head()


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3.0,0.0,"O'Donoghue, Ms. Bridget",female,NaN,0.0,0.0,364856,7.750,NaN,Q,NaN,NaN,NaN
1,2.0,0.0,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0.0,0.0,250655,26.000,NaN,S,NaN,NaN,NaN
2,2.0,1.0,"Smith, Miss. Marion Elsie",female,40.0,0.0,0.0,31418,13.000,NaN,S,9,NaN,NaN
3,3.0,1.0,"Goldsmith, Mrs. Frank John (Emily Alice Brown)",female,31.0,1.0,1.0,363291,20.525,NaN,S,C D,NaN,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,"McCoy, Miss. Agnes",female,NaN,2.0,0.0,367226,23.250,NaN,Q,16,NaN,NaN


In [2]:
columns_to_drop=["name","ticket","cabin","embarked"]
data_clean=data.drop(columns_to_drop,axis=1)
data_clean.head()

,pclass,survived,sex,age,sibsp,parch,fare,boat,body,home.dest
0,3.0,0.0,female,NaN,0.0,0.0,7.750,NaN,NaN,NaN
1,2.0,0.0,male,39.0,0.0,0.0,26.000,NaN,NaN,NaN
2,2.0,1.0,female,40.0,0.0,0.0,13.000,9,NaN,NaN
3,3.0,1.0,female,31.0,1.0,1.0,20.525,C D,NaN,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,female,NaN,2.0,0.0,23.250,16,NaN,NaN


In [3]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_clean["sex"]=le.fit_transform(data_clean["sex"])
data_clean.head()

,pclass,survived,sex,age,sibsp,parch,fare,boat,body,home.dest
0,3.0,0.0,0,NaN,0.0,0.0,7.750,NaN,NaN,NaN
1,2.0,0.0,1,39.0,0.0,0.0,26.000,NaN,NaN,NaN
2,2.0,1.0,0,40.0,0.0,0.0,13.000,9,NaN,NaN
3,3.0,1.0,0,31.0,1.0,1.0,20.525,C D,NaN,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,0,NaN,2.0,0.0,23.250,16,NaN,NaN


In [4]:
data_clean=data_clean.fillna(data_clean["age"].mean())
data_clean.head()

,pclass,survived,sex,age,sibsp,parch,fare,boat,body,home.dest
0,3.0,0.0,0,29.838978,0.0,0.0,7.750,29.839,29.838978,29.839
1,2.0,0.0,1,39.000000,0.0,0.0,26.000,29.839,29.838978,29.839
2,2.0,1.0,0,40.000000,0.0,0.0,13.000,9,29.838978,29.839
3,3.0,1.0,0,31.000000,1.0,1.0,20.525,C D,29.838978,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,0,29.838978,2.0,0.0,23.250,16,29.838978,29.839


In [5]:
input_cols=["pclass","sex","age","sibsp","parch","fare"]
output_cols=["survived"]
X=data_clean[input_cols]
Y=data_clean[output_cols]
Y.head()

,survived
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0


In [6]:
def entropy(col):
    counts=np.unique(col,return_counts=True)
    N=float(col.shape[0])
    ent=0.0
    for i in counts[1]:
        P=i/N
        ent+=(-1.0*P*np.log2(P))
    return ent

In [7]:
def information_gain(x_data,fkey,fval):
    left,right=divide_data(x_data,fkey,fval)
    l=float(left.shape[0])/x_data.shape[0]
    r=float(right.shape[0])/x_data.shape[0]
    if left.shape[0]==0 or right.shape[0]==0:
        return -1000000
    i_gain=entropy(x_data.survived)-(l*entropy(left.survived))+(r*entropy(right.survived))
    return i_gain

In [8]:
def divide_data(x_data,fkey,fval):
    x_right=pd.DataFrame([],columns=x_data.columns)
    x_left=pd.DataFrame([],columns=x_data.columns)
    for ix in range(x_data.shape[0]):
        val=x_data[fkey].loc[ix]
        if val>fval:
            x_right=x_right.append(x_data.loc[ix])
        else:
            x_left=x_left.append(x_data.loc[ix])
    return x_left,x_right


In [19]:
class DecisionTree:
    def __init__(self,depth=0,maxdepth=5):
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.maxdepth=maxdepth
        self.depth=depth
        self.target=None
        
    def train(self,X_train):
        features=["pclass","sex","age","sibsp","parch","fare"]
        info_gains=[]
        for ix in features:
            i_gain=information_gain(X_train,ix,X_train[ix].mean())
            info_gains.append(i_gain)
        self.fkey=features[np.argmax(info_gains)]
        self.fval=X_train[self.fkey].mean()
        print("making tree feature",self.fkey)
        data_left,data_right=divide_data(X_train,self.fkey,self.fval)
        data_left=data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
        if data_left.shape[0]==0 or data_right.shape[0]==0:
            if X_train.survived.mean()>=0.5:
                self.target="survive"
            else:
                self.target="dead"
            return   
      
              #if data_left.shape[0]==0 or data_right.shape[0]==0:
            if self.depth>=self.maxdepth: 
                if X_train.survived.mean()>=0.5:
                    self.target="survive"
                else:
                    self.target="dead"
                return
        self.left=DecisionTree(depth=self.depth+1,maxdepth=self.maxdepth)
        self.left.train(data_left)
        self.right=DecisionTree(depth=self.depth+1,maxdepth=self.maxdepth)
        self.right.train(data_right)
        #if data_left.shape[0]==0 or data_right.shape[0]==0:
        if X_train.survived.mean()>=0.5:
            self.target="survive"
        else:
            self.target="dead"
        return        
     
          
                
        
    def predict(self,test):
        if test[self.fkey]>self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)
            
            
        

In [20]:
split=int(0.7*data_clean.shape[0])
train_data=data_clean[:split]
test_data=data_clean[split:]
test_data=test_data.reset_index(drop=True)
dt=DecisionTree()
test_case=pd.read_csv("Test.csv")
dt.train(data_clean)

making tree feature sex
making tree feature pclass
making tree feature pclass
making tree feature sibsp
making tree feature age
making tree feature age
making tree feature age
making tree feature age
making tree feature age
making tree feature pclass
making tree feature parch
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature age
making tree feature age
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature fare
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature age
making tree feature age
making tree feature pclass
m

making tree feature fare
making tree feature age
making tree feature age
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature sibsp
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature age
making tree feature age
making tree feature sibsp
making tree feature parch
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature sibsp
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature age
making tree feature sibsp
making tree feature pc

making tree feature parch
making tree feature age
making tree feature age
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature parch
making tree feature age
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature age
making tree feature age
making tree feature age
making tree feature age
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature parch
making tree feature age
making tree feature age
mak

making tree feature fare
making tree feature fare
making tree feature age
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature age
making tree feature age
making tree feature age
making tree feature age
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature age
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
maki

making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature sibsp
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature sibsp
making tree feature fare
making tree feature fare
making tree feature fare
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature fare
making tree feature fare
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature sibsp
making t

making tree feature pclass
making tree feature fare
making tree feature age
making tree feature age
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature fare
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature fare
making tree feature pclass
making tree feature age
making tree feature age
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass
making tree feature parch
making tree feature fare
making tree feature age
making tree feature age
making tree feature age
making tree feature pclass
making tree feature age
making tree feature pclass
making tree feature pclass

In [21]:
test_case["sex"]=le.fit_transform(test_case["sex"])
print(dt.fkey)
print(dt.fval)
print(dt.left.fkey)
#print(dt.right.fkey)

sex
0.6422200198216056
pclass


In [22]:
test_case.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Flynn, Mr. John Irwin (""Irving"")",1,36.0,0.0,0.0,PC 17474,26.3875,E25,S,5,NaN,"Brooklyn, NY"
1,3.0,"Sage, Miss. Constance Gladys",0,NaN,8.0,2.0,CA. 2343,69.5500,NaN,S,NaN,NaN,NaN
2,1.0,"Rood, Mr. Hugh Roscoe",1,NaN,0.0,0.0,113767,50.0000,A32,S,NaN,NaN,"Seattle, WA"
3,2.0,"Gillespie, Mr. William Henry",1,34.0,0.0,0.0,12233,13.0000,NaN,S,NaN,NaN,"Vancouver, BC"
4,2.0,"Collander, Mr. Erik Gustaf",1,28.0,0.0,0.0,248740,13.0000,NaN,S,NaN,NaN,"Helsinki, Finland Ashtabula, Ohio"


In [27]:
test_case=test_case.fillna(test_case["age"].mean())
XX=test_case[input_cols]
y_pred=[]
for i in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[i]))
print(y_pred)

['dead', 'survive', 'dead', 'dead', 'dead', 'survive', 'survive', 'dead', 'dead', 'dead', 'dead', 'survive', 'dead', 'dead', 'survive', 'dead', 'survive', 'dead', 'dead', 'dead', 'dead', 'dead', 'survive', 'dead', 'dead', 'dead', 'dead', 'dead', 'survive', 'survive', 'survive', 'survive', 'dead', 'dead', 'dead', 'dead', 'dead', 'survive', 'dead', 'dead', 'survive', 'survive', 'dead', 'survive', 'dead', 'dead', 'survive', 'dead', 'survive', 'dead', 'dead', 'dead', 'dead', 'dead', 'dead', 'survive', 'survive', 'survive', 'dead', 'survive', 'dead', 'dead', 'dead', 'survive', 'survive', 'dead', 'dead', 'dead', 'dead', 'dead', 'dead', 'dead', 'dead', 'dead', 'survive', 'survive', 'dead', 'dead', 'dead', 'survive', 'survive', 'survive', 'survive', 'dead', 'survive', 'dead', 'dead', 'dead', 'dead', 'survive', 'survive', 'dead', 'survive', 'dead', 'dead', 'dead', 'survive', 'survive', 'survive', 'dead', 'dead', 'survive', 'dead', 'survive', 'survive', 'survive', 'survive', 'survive', 'dead', '

In [25]:
#print(XX.head())
from sklearn.tree import DecisionTreeClassifier
sk=DecisionTreeClassifier(criterion='entropy')
sk.fit(data_clean[input_cols],data_clean[output_cols])
sk.predict(test_case[input_cols])
sk.score(test_case[input_cols],test_case[output_cols])

KeyError: "None of [Index(['survived'], dtype='object')] are in the [columns]"

In [26]:
XX=test_case[input_cols]
y_pred=[]
for i in range(XX.shape[0]):
    y_pred.append(dt.predict(XX.loc[i]))
print(y_pred)

['survive', 'dead', 'dead', 'survive', 'dead', 'survive', 'survive', 'dead', 'dead', 'dead', 'dead', 'dead', 'dead', 'dead', 'survive', 'dead', 'dead', 'survive', 'survive', 'dead', 'survive', 'dead', 'survive', 'dead', 'dead', 'survive', 'dead', 'survive', 'survive', 'dead', 'dead', 'dead', 'dead', 'survive', 'dead', 'survive', 'dead', 'dead', 'dead', 'survive', 'survive', 'survive', 'dead', 'survive', 'dead', 'dead', 'survive', 'dead', 'dead', 'dead', 'survive', 'survive', 'survive', 'survive', 'dead', 'dead', 'dead', 'dead', 'survive', 'dead', 'survive', 'survive', 'survive', 'dead', 'survive', 'survive', 'dead', 'dead', 'dead', 'survive', 'survive', 'dead', 'survive', 'dead', 'survive', 'dead', 'survive', 'survive', 'dead', 'dead', 'survive', 'dead', 'dead', 'dead', 'dead', 'survive', 'dead', 'dead', 'survive', 'survive', 'survive', 'dead', 'dead', 'dead', 'survive', 'survive', 'survive', 'survive', 'survive', 'dead', 'survive', 'dead', 'dead', 'survive', 'dead', 'dead', 'survive',